<a href="https://colab.research.google.com/github/shelleygoel/streamflow-forecast/blob/main/collect_watershed_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

# Import libraries


In [1]:
from birdy import WPSClient

import datetime as dt
import json
import os
from pathlib import Path

import matplotlib.pyplot as plt
from netCDF4 import Dataset
import pandas as pd
import xarray as xr

# Canopex Watershed Discharge Data from Pavics

Pavics catalog https://pavics.ouranos.ca/twitcher/ows/proxy/thredds


In [2]:
# DATA MAIN SOURCE - DAP link to CANOPEX dataset
CANOPEX_DAP = "https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/ets/Watersheds_5797_cfcompliant.nc"

In [3]:
# Open Canopex dataset using DAP link
ds = xr.open_dataset(CANOPEX_DAP)
display(ds)

<xarray.Dataset>
Dimensions:        (time: 22280, watershed: 5797)
Coordinates:
  * time           (time) datetime64[ns] 1950-01-01 1950-01-02 ... 2010-12-31
  * watershed      (watershed) |S64 b'St. John River at Ninemile Bridge, Main...
Data variables:
    drainage_area  (watershed) float64 ...
    pr             (watershed, time) float64 ...
    tasmax         (watershed, time) float64 ...
    tasmin         (watershed, time) float64 ...
    discharge      (watershed, time) float64 ...
Attributes: (12/15)
    title:          Hydrometeorological data for lumped hydrological modellin...
    institute_id:   ETS
    contact:        Richard Arsenault: richard.arsenault@etsmtl.ca
    date_created:   2020-08-01
    source:         Hydrometric data from USGS National Water Information Ser...
    featureType:    timeSeries
    ...             ...
    activity:       PAVICS_Hydro
    Conventions:    CF-1.6, ACDD-1.3
    summary:        Hydrometeorological database for the PAVICS-Hydro platfor...
    institution:    ETS (École de technologie supérieure)
    DODS.strlen:    72
    DODS.dimName:   string72

 # Station Metadata
 

In [6]:
watersheds = ds.watershed.to_series()
watersheds

watershed
b'St. John River at Ninemile Bridge, Maine'                  b'St. John River at Ninemile Bridge, Maine'
b'St. John River at Dickey, Maine'                                    b'St. John River at Dickey, Maine'
b'Fish River near Fort Kent, Maine'                                  b'Fish River near Fort Kent, Maine'
b'St. John River below Fish R, nr Fort Kent, Maine'    b'St. John River below Fish R, nr Fort Kent, M...
b'Aroostook River near Masardis, Maine'                          b'Aroostook River near Masardis, Maine'
                                                                             ...                        
b'NITH RIVER ABOVE NITHBURG'                                                b'NITH RIVER ABOVE NITHBURG'
b'FAIRCHILD CREEK NEAR BRANTFORD'                                      b'FAIRCHILD CREEK NEAR BRANTFORD'
b'MIDDLE THAMES RIVER AT THAMESFORD'                                b'MIDDLE THAMES RIVER AT THAMESFORD'
b'BIG OTTER CREEK AT TILLSONBURG'            

In [7]:
metadata = pd.read_csv('STATION_METADATA.csv')
metadata

,CANOPEX_ID,STATION_ID,STATION_NAME,PROVINCE,STATION_LONGITUDE,STATION_LATITUDE,HYDROSHEDS_AREA,HYDAT_AREA,ACTIVE_STATION,RHBN_STATION,REAL_TIME_STATION,KÖPPEN-GEIGER ID,ENVCAN_NB_YEARS,ENVCAN_FIRST_YEAR,ENVCAN_LAST_YEAR,NRCAN_NB_YEARS,NRCAN_FIRST_YEAR,NRCAN_LAST_YEAR
0,1,'06CD002','CHURCHILL RIVER ABOVE OTTER RAPIDS','SK',-104.735832,55.647499,114248.000,119000.00,'True','True','True','Dfb',50,1963,2012,48,1963,2010
1,2,'05OH007','SEINE RIVER NEAR STE. ANNE','MB',-96.609032,49.643639,702.093,580.00,'True','True','True','Dfb',49,1964,2012,47,1964,2010
2,3,'10AD001','HYLAND RIVER NEAR LOWER POST','BC',-128.150833,59.950829,9148.050,9450.00,'True','False','False','Dfc',43,1951,1993,44,1950,1993
3,4,'10BC001','COAL RIVER AT THE MOUTH','BC',-126.950562,59.691391,8775.390,9190.00,'True','False','True','Dfc',35,1961,1995,35,1961,1995
4,5,'09AA013','TUTSHI RIVER AT OUTLET OF TUTSHI LAKE','BC',-134.332504,59.947781,895.434,989.00,'True','False','True','ET',57,1956,2012,55,1956,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,694,'02GB007','FAIRCHILD CREEK NEAR BRANTFORD','ON',-80.150000,43.150000,380.000,388.64,'False','False','False','Dfb',49,1964,2012,47,1964,2010
694,695,'02GD004','MIDDLE THAMES RIVER AT THAMESFORD','ON',-80.990000,43.060000,292.000,306.00,'False','False','False','Dfb',62,1951,2012,61,1950,2010
695,696,'02GD006','THAMES RIVER NEAR INGERSOLL','ON',-80.930000,43.020000,563.000,554.00,'False','False','False','Dfb',7,1951,1957,8,1950,1957
696,697,'02GC010','BIG OTTER CREEK AT TILLSONBURG','ON',-80.720000,42.860000,378.000,354.10,'True','False','False','Dfb',53,1960,2012,51,1960,2010


## Cleanup Station Data

In [8]:
metadata.columns = [col.lower() for col in metadata.columns]

In [9]:
metadata['station_name'] = metadata['station_name'].map(lambda s: s[1:-1].encode('utf-8'))
metadata['station_id'] = metadata['station_id'].map(lambda s: s[1:-1].encode('utf-8'))
metadata.head()

,canopex_id,station_id,station_name,province,station_longitude,station_latitude,hydrosheds_area,hydat_area,active_station,rhbn_station,real_time_station,köppen-geiger id,envcan_nb_years,envcan_first_year,envcan_last_year,nrcan_nb_years,nrcan_first_year,nrcan_last_year
0,1,b'06CD002',b'CHURCHILL RIVER ABOVE OTTER RAPIDS','SK',-104.735832,55.647499,114248.000,119000.0,'True','True','True','Dfb',50,1963,2012,48,1963,2010
1,2,b'05OH007',b'SEINE RIVER NEAR STE. ANNE','MB',-96.609032,49.643639,702.093,580.0,'True','True','True','Dfb',49,1964,2012,47,1964,2010
2,3,b'10AD001',b'HYLAND RIVER NEAR LOWER POST','BC',-128.150833,59.950829,9148.050,9450.0,'True','False','False','Dfc',43,1951,1993,44,1950,1993
3,4,b'10BC001',b'COAL RIVER AT THE MOUTH','BC',-126.950562,59.691391,8775.390,9190.0,'True','False','True','Dfc',35,1961,1995,35,1961,1995
4,5,b'09AA013',b'TUTSHI RIVER AT OUTLET OF TUTSHI LAKE','BC',-134.332504,59.947781,895.434,989.0,'True','False','True','ET',57,1956,2012,55,1956,2010


In [43]:
ws_names = metadata['station_name']
ws_names = list(set(ws_names) & set(watersheds))
assert len(ws_names) == 514
assert b'SIKANNI CHIEF RIVER NEAR FORT NELSON' in ws_names

ws1 = ds.sel(watershed=ws_names)
ws1

<xarray.Dataset>
Dimensions:        (time: 22280, watershed: 514)
Coordinates:
  * time           (time) datetime64[ns] 1950-01-01 1950-01-02 ... 2010-12-31
  * watershed      (watershed) |S64 b'RUSHLAKE CREEK ABOVE HIGHFIELD RESERVOI...
Data variables:
    drainage_area  (watershed) float64 325.0 1.7e+03 1.48e+03 ... 1.84e+04 454.0
    pr             (watershed, time) float64 ...
    tasmax         (watershed, time) float64 ...
    tasmin         (watershed, time) float64 ...
    discharge      (watershed, time) float64 ...
Attributes: (12/15)
    title:          Hydrometeorological data for lumped hydrological modellin...
    institute_id:   ETS
    contact:        Richard Arsenault: richard.arsenault@etsmtl.ca
    date_created:   2020-08-01
    source:         Hydrometric data from USGS National Water Information Ser...
    featureType:    timeSeries
    ...             ...
    activity:       PAVICS_Hydro
    Conventions:    CF-1.6, ACDD-1.3
    summary:        Hydrometeorological database for the PAVICS-Hydro platfor...
    institution:    ETS (École de technologie supérieure)
    DODS.strlen:    72
    DODS.dimName:   string72

In [ ]:
metadata.info()

In [ ]:
ws_names[0]

In [44]:
metadata_filter = metadata.set_index('station_name').loc[ws_names]
assert metadata_filter.shape == (514, 17)

# Watershed properties using PAVICS

In [20]:
# Set environment variable WPS_URL to "http://localhost:9099" to run on the default local server
url = os.environ.get("WPS_URL")
if not url:
    url = "https://pavics.ouranos.ca/twitcher/ows/proxy/raven/wps"

wps = WPSClient(url)

In [47]:
elevation = []
slope = []
aspect = []
for name, station in metadata.head(5).iterrows():

    long = station['station_longitude']
    lat = station['station_latitude']
    coords = f'{long}, {lat}'
    
    select_resp = wps.hydrobasins_select(
        location=coords, aggregate_upstream=False
    )
    feature_url, upstream_basins_url = select_resp.get(asobj=False)
    terrain_resp = wps.terrain_analysis(
        shape=feature_url, select_all_touching=True, projected_crs=3978
    )

    properties, dem = terrain_resp.get(asobj=True)

    elevation.append(properties[0]["elevation"])
    slope.append(properties[0]["slope"])
    aspect.append(properties[0]["aspect"])

In [48]:
elevation

[408.80132668423033,
 267.7578172544701,
 642.9332419123606,
 648.9552285210902,
 808.9434451466537]

In [49]:
slope

[2.773313674683007,
 0.3571142322097378,
 3.3022659656613373,
 6.045237586768188,
 5.289324744645429]

In [50]:
aspect

[79.22662560900149,
 293.9389726560986,
 230.06234886036535,
 231.81917420369305,
 42.7311593946337]

In [53]:
metadata_filter.head()

,canopex_id,station_id,province,station_longitude,station_latitude,hydrosheds_area,hydat_area,active_station,rhbn_station,real_time_station,köppen-geiger id,envcan_nb_years,envcan_first_year,envcan_last_year,nrcan_nb_years,nrcan_first_year,nrcan_last_year
station_name,,,,,,,,,,,,,,,,,
b'RUSHLAKE CREEK ABOVE HIGHFIELD RESERVOIR',604,b'05JC004','SK',-107.470000,50.250000,306.00,325.0,'True','False','False','BSk',48,1965,2012,46,1965,2010
b'BLUE RIVER NEAR THE MOUTH',12,b'10AC004','BC',-129.127777,59.758331,1542.13,1700.0,'True','True','True','Dfc',33,1963,1995,33,1963,1995
b'ST. MARY RIVER NEAR MARYSVILLE',278,b'08NG046','BC',-116.168610,49.608059,1475.72,1480.0,'True','False','False','Dfc',45,1951,1995,46,1950,1995
b'BRIDGE CREEK AT GULL LAKE',606,b'05HA015','SK',-108.490000,50.090000,375.00,379.0,'True','False','True','Dfb',62,1951,2012,61,1950,2010
b'ODEI RIVER NEAR THOMPSON',69,b'05TG003','MB',-97.357620,55.995258,5817.02,6110.0,'True','False','True','Dfc',34,1979,2012,32,1979,2010


In [54]:
from ravenpy.utilities.testdata import get_file


tmp = pd.read_csv(get_file("regionalisation_data/gauged_catchment_properties.csv"))

In [57]:
s1 = tmp[tmp['StationID'] == metadata_filter.iloc[4]['station_id'].decode()]

In [59]:
s1

,Unnamed: 0,ID,StationID,area,latitude,longitude,gravelius,perimeter,RunSuccessShape,elevation,...,RunSuccessTerrain,forest,grass,wetland,water,urban,shrubs,crops,snowIce,RunSuccessLandUse
5317,5317,5318.0,05TG003,6002.862827,56.166195,-97.952597,2.426774,666520.501791,1.0,255.210935,...,1.0,0.5375,0.017851,0.118111,0.07974,0.000551,0.246246,0.0,0.0,1.0


In [60]:
s1[['StationID', 'elevation', 'slope', 'aspect']]

,StationID,elevation,slope,aspect
5317,05TG003,255.210935,1.004718,158.074019


In [ ]:
import geopandas as gpd
df = gpd.read_file(feature_url)
display(df)
df.plot()